In [ ]:
# Poetic Personality Chatbot for Google Colab
# Save as: poetic_personality_chatbot.ipynb

# Install required packages
pip install -q google-generativeai
pip install -q ipywidgets

import time
import random
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import userdata

# Configure Gemini API (using Colab Secrets)
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-pro')
    print("✅ Gemini API connected successfully")
except (KeyError, ValueError) as e:
    print(f"⚠️ API key error: {e}. Using built-in poetry mode.")
    model = None

# Emotion detection
class EmotionDetector:
    EMOTION_KEYWORDS = {
        'happy': ['happy', 'joy', 'excited', 'great', 'amazing'],
        'sad': ['sad', 'cry', 'hurt', 'pain', 'lonely'],
        'angry': ['angry', 'mad', 'hate', 'annoyed', 'frustrated'],
        'love': ['love', 'romance', 'heart', 'crush', 'affection'],
        'hopeful': ['hope', 'dream', 'future', 'wish', 'believe']
    }

    @classmethod
    def detect_emotion(cls, text):
        text_lower = text.lower()
        emotion_scores = {}
        for emotion, keywords in cls.EMOTION_KEYWORDS.items():
            score = sum(1 for keyword in keywords if keyword in text_lower)
            if score > 0:
                emotion_scores[emotion] = score
        return max(emotion_scores, key=emotion_scores.get) if emotion_scores else 'neutral'

# Poetry generator
class PoetryGenerator:
    POETRY_TEMPLATES = {
        'happy': [
            "Sunshine dances in your heart,\nJoy bubbles up, a brand new start.\nYour smile lights up the world so bright,\nFilling everyone with pure delight! ✨",
            "Laughter echoes in the air,\nHappiness beyond compare.\nLike flowers blooming in the spring,\nYour joy makes all the angels sing! 🌸"
        ],
        'sad': [
            "Though clouds may gather in your sky,\nRemember stars still shine up high.\nThis gentle rain will help you grow,\nAnd soon the sun will start to show. 🌧️",
            "In valleys deep, the flowers rest,\nTo bloom again when spring is blessed.\nYour tears today will water dreams,\nThat tomorrow brightly gleams. 🌱"
        ],
        # Other emotion templates...
        'neutral': [
            "In quiet moments, beauty hides,\nWhere peaceful wonder softly glides.\nLife's gentle rhythm, calm and true,\nFinds poetry in me and you. 🍃"
        ]
    }

    @classmethod
    def generate_poem(cls, emotion, user_input):
        poems = cls.POETRY_TEMPLATES.get(emotion, cls.POETRY_TEMPLATES['neutral'])
        return random.choice(poems)

# Typing animation effect
def typewriter_effect(text, delay=0.05):
    output_area = widgets.Output()
    display(output_area)
    with output_area:
        for char in text:
            print(char, end='', flush=True)
            time.sleep(delay)
        print()
    return output_area

# Chatbot core
class PoeticChatbot:
    def __init__(self):
        self.conversation_history = []

    def generate_response(self, user_input):
        emotion = EmotionDetector.detect_emotion(user_input)
        print(f"💭 Detected emotion: {emotion}")

        if model:
            print("🌸 Composing personalized poem with AI...")
            prompt = f"""
            You are a poetic soul. Transform this message into a 4-line poem with:
            1. ABAB rhyme scheme
            2. Nature metaphors
            3. Emotional reflection of: {emotion}
            4. Uplifting resolution

            Message: "{user_input}"
            """
            try:
                response = model.generate_content(prompt)
                return response.text
            except Exception as e:
                print(f"⚠️ AI error: {e}. Using built-in poetry.")

        return PoetryGenerator.generate_poem(emotion, user_input)

# Main chat interface
def run_chatbot():
    chatbot = PoeticChatbot()
    print("🌸" + "="*60 + "🌸")
    print("       Welcome to the Poetic Soul Chatbot")
    print("     Your emotions transformed into verses")
    print("🌸" + "="*60 + "🌸")
    print("\nType your feelings ('quit' to exit)\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'quit':
            print("\n🌙 Until our verses meet again...")
            break

        response = chatbot.generate_response(user_input)
        print("\nBot: ", end='')
        typewriter_effect(response)
        print()

# Run the chatbot
if __name__ == "__main__":
    run_chatbot()
